## Notebook Setup 
The following cell will install Drake, checkout the underactuated repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.
- On Binder, the machines should already be provisioned by the time you can run this; it should return (almost) instantly.

More details are available [here](http://underactuated.mit.edu/underactuated.html?chapter=drake).

In [ ]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Global optimization with Sums-of-Squares (the "Six Hump Camel")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pydrake.all import MathematicalProgram, Solve

In [ ]:
# Setup SOS program and define indeterminate variables.
prog = MathematicalProgram()
x = prog.NewIndeterminates(2, 'x')

# This is the famous "six-hump camel back function".
# It has six local minima, two of them being global minima.
p = 4 * x[0]**2 + x[0] * x[1] - 4 * x[1]**2 - 2.1 * x[0]**4 + 4 * x[1]**4 + x[0]**6 / 3

# Find the minimum value by adding the SOS constraint p(x) - lam is SOS.
lam = prog.NewContinuousVariables(1, 'lam')[0]
prog.AddSosConstraint(p - lam)

# Maximize lambda.
prog.AddCost(- lam)

# Solve SOS.
result = Solve(prog)
assert result.is_success()

In [ ]:
# Now, let's plot it.
fig = plt.figure(figsize=(6, 10))
ax0 = fig.add_subplot(211, projection='3d')
ax1 = fig.add_subplot(212)
x0s = np.linspace(-2.2, 2.2, 301)
x1s = np.linspace(-1.2, 1.2, 301)
[X0, X1] = np.meshgrid(x0s, x1s)
P = X0.copy()
for i in range(len(x0s)):
    for j in range(len(x1s)):
        P[i, j] = p.Evaluate({x[0]: X0[i, j], x[1]: X1[i, j]})
ax0.plot_surface(X0, X1, P)
ax0.set_xlabel(r'$x_1$')
ax0.set_ylabel(r'$x_2$')
ax1.contour(X0, X1, P, 100)

print(f'Minimum from SOS: {result.GetSolution(lam)}')
print(f'Minimum sampled value: {np.min(P)}')